In [5]:
import pandas as pd
import numpy as np

df = pd.read_excel('df_wide_for_regwt.xlsx', index_col=1)
print(df)

result = []

           pref  prop_15_64  prop_65_ov  prop_second_shot  variant  interval1  \
intv                                                                            
1         Tokyo    0.657855    0.230515          0.413359       54   3.708521   
0        Toyama    0.562261    0.322797          0.441894       20   0.162043   
0      Ishikawa    0.580844    0.296134          0.471494        9   1.225604   
0         Fukui    0.567708    0.305990          0.493298        0   0.207550   
0          Gifu    0.573729    0.301459          0.472271       14   1.283152   
0         Aichi    0.618247    0.250530          0.397027       17   1.242712   
0           Mie    0.580573    0.297586          0.436981       37   0.871775   
0         Shiga    0.602546    0.260255          0.424260        8   0.989035   
0         Kyoto    0.592722    0.291521          0.405815       22   1.568381   
0         Osaka    0.605404    0.276308          0.404178       15   2.004785   
0         Hyogo    0.585803 

In [8]:
def Reg_Wt_SCM(intv,df):
    
    # Intervention prefecture
    x1 = np.matrix(df[df.pref==intv].loc[:,'prop_15_64':'interval8'].values.tolist()).T
    x1 = np.insert(x1,0,1,axis=0)
    y1 = np.matrix(df[df.pref==intv].loc[:,'y173':'y242'].values.tolist()).T
    
    # Other prefecture (Control pool)
    df = df[df.pref!="Tokyo"]
    x0 = np.matrix(df[df.pref!=intv].loc[:,'prop_15_64':'interval8'].values.tolist()).T
    x0 = np.insert(x0,0,1,axis=0)
    y0 = np.matrix(df[df.pref!=intv].loc[:,'y173':'y242'].values.tolist()).T
    
    print("x1.shape", x1.shape)
    print("x0.shape", x0.shape)
    print("y1.shape", y1.shape)
    print("y0.shape", y0.shape)
    
    
    ## Other matrix
    iota   = np.matrix(np.ones(29),dtype=float).T
    
    # Regression weight
    w_reg  = np.dot(np.dot(x0.T,np.linalg.inv(np.dot(x0,x0.T))),x1)
    type(w_reg)
    
    # sum_of_weight = np.dot(iota.T,w_reg)
    sum_of_weight = np.sum(w_reg)
    print('sum_of_weight = ', sum_of_weight) # sum_of_weight =  [[1.]]があればOK
    
    # Calculate pre-intervention values of synthetic control
    syn_cntr_x = np.dot(x0,w_reg)
    # print('Synthetic Control of Pre_trt\n', syn_cntr_x)
    
    # Pre-RMSPE
    mat_pre_diff1 = x1 - syn_cntr_x   # 人口割合等も含む縦ベクトル
    mat_pre_diff2 = mat_pre_diff1[5:13,:] # 感染者数の差のみの縦ベクトル

    mat_pre_diff1sq = np.square(mat_pre_diff1) # 2乗
    mat_pre_diff2sq = np.square(mat_pre_diff2)

    pre_rmspe1 = np.sqrt(np.sum(mat_pre_diff1sq)/len(mat_pre_diff1sq)) # 平均をの平方根
    pre_rmspe2 = np.sqrt(np.sum(mat_pre_diff2sq)/len(mat_pre_diff2sq))
    
    # Calculate post-intervention values of synthetic control
    syn_cntr_y = np.dot(y0, w_reg)

    # Post-RMSPE
    mat_post_diff = y1 - syn_cntr_y

    mat_post_diffsq = np.square(mat_post_diff) # 2乗

    post_rmspe = np.sqrt(np.sum(mat_post_diffsq)/len(mat_post_diffsq))

    # Output
    print("Tokyo")
    print("Pre-RMSPE =", pre_rmspe2)
    print("Post-RMSPE=", post_rmspe)
    
    result = [intv, sum_of_weight, pre_rmspe2, post_rmspe]

    return result

In [9]:
Reg_Wt_SCM("Tokyo", df)

x1.shape (13, 1)
x0.shape (13, 29)
y1.shape (70, 1)
y0.shape (70, 29)
sum_of_weight =  0.9999999999992415
Tokyo
Pre-RMSPE = 1.4377044981574552e-12
Post-RMSPE= 7.412323089976295


['Tokyo', 0.9999999999992415, 1.4377044981574552e-12, 7.412323089976295]

In [10]:
Reg_Wt_SCM("Toyama",df)

x1.shape (13, 1)
x0.shape (13, 28)
y1.shape (70, 1)
y0.shape (70, 28)
sum_of_weight =  0.9999999999998324
Tokyo
Pre-RMSPE = 2.28792947537344e-13
Post-RMSPE= 2.7221533916835767


['Toyama', 0.9999999999998324, 2.28792947537344e-13, 2.7221533916835767]

In [14]:
result.append(Reg_Wt_SCM("Tokyo", df))
result

x1.shape (13, 1)
x0.shape (13, 29)
y1.shape (70, 1)
y0.shape (70, 29)
sum_of_weight =  0.9999999999992415
Tokyo
Pre-RMSPE = 1.4377044981574552e-12
Post-RMSPE= 7.412323089976295


[['Tokyo', 0.9999999999992415, 1.4377044981574552e-12, 7.412323089976295]]

In [15]:
control_pool = ["Toyama", "Ishikawa", "Fukui", "Gifu", "Aichi", "Mie", "Shiga", "Kyoto", "Osaka", "Hyogo", "Nara", "Wakayama", "Tottori", "Shimane", "Okayama", "Hiroshima", "Yamaguchi", "Tokushima", "Kagawa", "Ehime", "Kochi", "Fukuoka", "Saga", "Nagasaki", "Kumamoto", "Oita", "Miyazaki", "Kagoshima", "Okinawa"]

In [17]:
for x in control_pool:
    result.append(Reg_Wt_SCM(x,df))

x1.shape (13, 1)
x0.shape (13, 28)
y1.shape (70, 1)
y0.shape (70, 28)
sum_of_weight =  0.9999999999998324
Tokyo
Pre-RMSPE = 2.28792947537344e-13
Post-RMSPE= 2.7221533916835767
x1.shape (13, 1)
x0.shape (13, 28)
y1.shape (70, 1)
y0.shape (70, 28)
sum_of_weight =  1.0000000000031846
Tokyo
Pre-RMSPE = 5.8397993426108e-12
Post-RMSPE= 7.047861501428773
x1.shape (13, 1)
x0.shape (13, 28)
y1.shape (70, 1)
y0.shape (70, 28)
sum_of_weight =  0.9999999999995024
Tokyo
Pre-RMSPE = 1.2011551266415396e-12
Post-RMSPE= 3.3953390404083796
x1.shape (13, 1)
x0.shape (13, 28)
y1.shape (70, 1)
y0.shape (70, 28)
sum_of_weight =  1.0000000000002984
Tokyo
Pre-RMSPE = 7.879990653214412e-13
Post-RMSPE= 1.8432725938876664
x1.shape (13, 1)
x0.shape (13, 28)
y1.shape (70, 1)
y0.shape (70, 28)
sum_of_weight =  1.000000000002247
Tokyo
Pre-RMSPE = 3.921833922225646e-12
Post-RMSPE= 4.270087441468528
x1.shape (13, 1)
x0.shape (13, 28)
y1.shape (70, 1)
y0.shape (70, 28)
sum_of_weight =  0.9999999999982305
Tokyo
Pre-RMSP

In [18]:
result

[['Tokyo', 0.9999999999992415, 1.4377044981574552e-12, 7.412323089976295],
 ['Toyama', 0.9999999999998324, 2.28792947537344e-13, 2.7221533916835767],
 ['Ishikawa', 1.0000000000031846, 5.8397993426108e-12, 7.047861501428773],
 ['Fukui', 0.9999999999995024, 1.2011551266415396e-12, 3.3953390404083796],
 ['Gifu', 1.0000000000002984, 7.879990653214412e-13, 1.8432725938876664],
 ['Aichi', 1.000000000002247, 3.921833922225646e-12, 4.270087441468528],
 ['Mie', 0.9999999999982305, 3.2449804952923153e-12, 4.602499659568848],
 ['Shiga', 0.9999999999994228, 1.198044210967335e-12, 2.5568190530144443],
 ['Kyoto', 1.0000000000001998, 3.470669919074204e-13, 3.2835013743182326],
 ['Osaka', 0.9999999999957869, 7.283223945626258e-12, 2.911682261720389],
 ['Hyogo', 1.0000000000006892, 1.3235590835547014e-12, 1.926983674196783],
 ['Nara', 1.0000000000005786, 9.67187838083422e-13, 1.4706831967873686],
 ['Wakayama', 0.9999999999986754, 2.4375005354418805e-12, 2.2516683159776365],
 ['Tottori', 1.0000000000014

In [19]:
2/30

0.06666666666666667

In [20]:
output = pd.DataFrame(result)
output.to_csv('RegWt_SCM2_result.csv')